<a href="https://colab.research.google.com/github/Gaurav-Ramachandra/AI-Lab/blob/main/AI%20Lab9%203-12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: convert logic statement to cnf and then perform resolution in first order logic

from sympy.logic.boolalg import to_cnf, simplify_logic
from sympy import symbols, Or, And, Not

def resolution(clauses):
    """
    Performs resolution on a set of clauses in Conjunctive Normal Form (CNF).

    Args:
        clauses: A list of clauses, where each clause is a list of literals.
                 A literal is a symbol or its negation (represented by ~symbol).

    Returns:
        True if the set of clauses is unsatisfiable, False otherwise.
    """
    new_clauses = set()
    while True:
      for i in range(len(clauses)):
        for j in range(i+1, len(clauses)):
          resolvents = resolve(clauses[i],clauses[j])
          for resolvent in resolvents:
            if resolvent == []:
                return True  # Empty clause means unsatisfiable
            if not resolvent in clauses and not resolvent in new_clauses:
                new_clauses.add(tuple(resolvent))
      if not new_clauses:
        return False
      clauses.extend(list(new_clauses))
      new_clauses = set()


def resolve(clause1, clause2):
    result = []
    for literal1 in clause1:
        for literal2 in clause2:
            if literal1 == ~literal2:
                resolvent = list(set(clause1) - {literal1}) + list(set(clause2) - {literal2})
                result.append(resolvent)
    return result

# Example usage:
# Define the symbols
p, q, r = symbols('p q r')

# Original logical statement (example)
statement = Not(p) | (q & r)

# Convert to CNF
cnf_statement = to_cnf(statement)
print(f"CNF of the statement: {cnf_statement}")

# Convert CNF to list of clauses
# Note: this assumes a simple CNF structure
# In more complex cases, you might need more advanced parsing
clauses = []
cnf_statement_simplified = simplify_logic(cnf_statement)
if isinstance(cnf_statement_simplified, Or):
  for clause in cnf_statement_simplified.args:
      if isinstance(clause, And):
        clauses.append(list(clause.args))
      else:
        clauses.append([clause])
else:
  clauses = [[cnf_statement_simplified]]

# Convert symbols to literals (representation for resolution)
def to_literals(clauses):
    literals_clauses = []
    for clause in clauses:
        new_clause = []
        for literal in clause:
            if isinstance(literal, Not):
                new_clause.append(~literal.args[0])
            else:
                new_clause.append(literal)
        literals_clauses.append(new_clause)
    return literals_clauses

# Perform resolution
clauses = to_literals(clauses)
unsatisfiable = resolution(clauses)
print(f"Is the set of clauses unsatisfiable? {unsatisfiable}")

CNF of the statement: (q | ~p) & (r | ~p)
Is the set of clauses unsatisfiable? False


In [ ]:

class Clause:
    def __init__(self, literals):
        self.literals = set(literals)

    def __repr__(self):
        return " ∨ ".join(sorted(self.literals))

    def resolve(self, other):
        resolvents = []
        for literal in self.literals:
            negated_literal = f"¬{literal}" if not literal.startswith('¬') else literal[1:]
            if negated_literal in other.literals:
                new_literals = (self.literals - {literal}) | (other.literals - {negated_literal})
                resolvents.append(Clause(new_literals))
        return resolvents


def resolution(clauses, query):
    negated_query = Clause([f"¬{query}"])
    clauses.append(negated_query)

    new = set()
    seen_pairs = set()

    while True:
        pairs = [(clauses[i], clauses[j]) for i in range(len(clauses)) for j in range(i + 1, len(clauses))]
        for ci, cj in pairs:
            if (ci, cj) in seen_pairs or (cj, ci) in seen_pairs:
                continue
            seen_pairs.add((ci, cj))

            resolvents = ci.resolve(cj)
            for resolvent in resolvents:
                if not resolvent.literals:
                    return True
                new.add(frozenset(resolvent.literals))

        if new.issubset(set(map(frozenset, (c.literals for c in clauses)))):
            return False
        clauses.extend(Clause(list(literals)) for literals in new - set(map(frozenset, (c.literals for c in clauses))))
        new.clear()


KB = [
    Clause(["¬Food(Peanuts)", "Likes(John, Peanuts)"]),  # John likes all kinds of food
    Clause(["Food(Apple)"]),
    Clause(["Food(Vegetables)"]),
    Clause(["Food(Peanuts)"]),  # Explicit Peanuts as Food
    Clause(["¬Eats(Anil, Peanuts)", "Food(Peanuts)"]),
    Clause(["Eats(Anil, Peanuts)"]),  # Anil eats peanuts
    Clause(["Alive(Anil)"]),           # Anil is alive
    Clause(["¬Alive(Anil)", "¬Killed(Anil)"]),  # Alive -> Not Killed
    Clause(["Killed(Anil)", "Alive(Anil)"])     # Not Killed -> Alive
]

# Query: John likes peanuts
query = "Likes(John, Peanuts)"

# Run the resolution algorithm
if resolution(KB, query):
    print(f"The conclusion '{query}' is proven by resolution.")
else:
    print(f"The conclusion '{query}' cannot be proven.")


The conclusion 'Likes(John, Peanuts)' is proven by resolution.


In [ ]:
# Step 1: Helper function to negate a literal
def negate(literal):
   """Return the negation of a literal."""
   if isinstance(literal, tuple) and literal[0] == "not":
       return literal[1]
   else:
       return ("not", literal)




# Function to resolve two clauses
def resolve(clause1, clause2):
   """Return the resolvent of two clauses."""
   resolvents = set()
   for literal1 in clause1:
       for literal2 in clause2:
           if literal1 == negate(literal2):
               resolvent = (clause1 - {literal1}) | (clause2 - {literal2})
               print(f"    Resolving literal: {literal1} with {literal2}")
               print(f"    Resulting Resolvent: {resolvent}")
               resolvents.add(frozenset(resolvent))
   return resolvents




# Function to perform resolution on the KB and query with detailed output
def resolution_algorithm(KB, query):
   """Perform the resolution algorithm to check if the query can be proven."""
   print("\n--- Step-by-Step Resolution Process ---")
   # Add the negation of the query to the knowledge base
   negated_query = negate(query)
   KB.append(frozenset([negated_query]))
   print(f"Negated Query Added to KB: {negated_query}")




   # Initialize the set of clauses to process
   clauses = set(KB)




   step = 1
   while True:
       new_clauses = set()
       print(f"\nStep {step}: Resolving Clauses")
       for c1 in clauses:
           for c2 in clauses:
               if c1 != c2:
                   print(f"  Resolving clauses: {c1} and {c2}")
                   resolvent = resolve(c1, c2)
                   for res in resolvent:
                       if frozenset([]) in resolvent:
                           print("\nEmpty clause derived! The query is provable.")
                           return True  # Empty clause found, contradiction, query is provable
                       new_clauses.add(res)




       if new_clauses.issubset(clauses):
           print("\nNo new clauses can be derived. The query is not provable.")
           return False  # No new clauses, query is not provable




       clauses.update(new_clauses)
       step += 1




# Knowledge Base (KB) from the image facts
KB = [
   frozenset([("not", "food(x)"), ("likes", "John", "x")]),  # 1
   frozenset([("food", "Apple")]),                           # 2
   frozenset([("food", "vegetables")]),                      # 3
   frozenset([("not", "eats(y, z)"), ("killed", "y"), ("food", "z")]),  # 4
   frozenset([("eats", "Anil", "Peanuts")]),                 # 5
   frozenset([("alive", "Anil")]),                           # 6
   frozenset([("not", "eats(Anil, w)"), ("eats", "Harry", "w")]),  # 7
   frozenset([("killed", "g"), ("alive", "g")]),             # 8
   frozenset([("not", "alive(k)"), ("not", "killed(k)")]),   # 9
   frozenset([("likes", "John", "Peanuts")])                 # 10
]




# Query to prove
query = ("likes", "John", "Peanuts")




# Perform resolution to check if the query is provable
result = resolution_algorithm(KB, query)
if result:
   print("\nQuery is provable.")
else:
   print("\nQuery is not provable.")


--- Step-by-Step Resolution Process ---
Negated Query Added to KB: ('not', ('likes', 'John', 'Peanuts'))

Step 1: Resolving Clauses
  Resolving clauses: frozenset({('not', ('likes', 'John', 'Peanuts'))}) and frozenset({('killed', 'y'), ('food', 'z'), ('not', 'eats(y, z)')})
  Resolving clauses: frozenset({('not', ('likes', 'John', 'Peanuts'))}) and frozenset({('food', 'Apple')})
  Resolving clauses: frozenset({('not', ('likes', 'John', 'Peanuts'))}) and frozenset({('eats', 'Anil', 'Peanuts')})
  Resolving clauses: frozenset({('not', ('likes', 'John', 'Peanuts'))}) and frozenset({('not', 'killed(k)'), ('not', 'alive(k)')})
  Resolving clauses: frozenset({('not', ('likes', 'John', 'Peanuts'))}) and frozenset({('not', 'eats(Anil, w)'), ('eats', 'Harry', 'w')})
  Resolving clauses: frozenset({('not', ('likes', 'John', 'Peanuts'))}) and frozenset({('likes', 'John', 'Peanuts')})
    Resolving literal: ('not', ('likes', 'John', 'Peanuts')) with ('likes', 'John', 'Peanuts')
    Resulting Reso

In [ ]:
import re
from typing import List, Dict, Set, Union

class Literal:
    def __init__(self, predicate: str, terms: List[str], negated: bool = False):
        """
        Represents a logical literal
        :param predicate: Name of the predicate
        :param terms: List of terms (arguments)
        :param negated: Boolean indicating if the literal is negated
        """
        self.predicate = predicate
        self.terms = terms
        self.negated = negated

    def __repr__(self):
        """String representation of the literal"""
        prefix = "¬" if self.negated else ""
        return f"{prefix}{self.predicate}({','.join(self.terms)})"

    def __eq__(self, other):
        """Check if two literals are equal"""
        return (self.predicate == other.predicate and
                self.terms == other.terms and
                self.negated == other.negated)

    def __hash__(self):
        """Allow use in sets"""
        return hash((self.predicate, tuple(self.terms), self.negated))

class Clause:
    def __init__(self, literals: Set[Literal]):
        """
        Represents a clause (disjunction of literals)
        :param literals: Set of Literal objects
        """
        self.literals = literals

    def __repr__(self):
        """String representation of the clause"""
        return " ∨ ".join(str(lit) for lit in self.literals)

    def __eq__(self, other):
        """Check if two clauses are equal"""
        return self.literals == other.literals

    def __hash__(self):
        """Allow use in sets"""
        return hash(frozenset(self.literals))

class CNFConverter:
    @staticmethod
    def eliminate_implications(formula: str) -> str:
        """
        Eliminate implications in the logical formula
        :param formula: Input logical formula
        :return: Formula without implications
        """
        # Replace implications (A -> B) with (¬A ∨ B)
        while '->' in formula:
            formula = re.sub(r'\(([^()]+)\s*->\s*([^()]+)\)', r'(¬\1 ∨ \2)', formula)
        return formula

    @staticmethod
    def move_negations_inward(formula: str) -> str:
        """
        Move negations inward using De Morgan's laws
        :param formula: Input logical formula
        :return: Formula with negations moved inward
        """
        # ¬(A ∧ B) becomes (¬A ∨ ¬B)
        formula = re.sub(r'¬\((.*?)\s*∧\s*(.*?)\)', r'(¬\1 ∨ ¬\2)', formula)
        # ¬(A ∨ B) becomes (¬A ∧ ¬B)
        formula = re.sub(r'¬\((.*?)\s*∨\s*(.*?)\)', r'(¬\1 ∧ ¬\2)', formula)
        return formula

    @staticmethod
    def distribute_or(formula: str) -> str:
        """
        Distribute OR over AND to convert to CNF
        :param formula: Input logical formula
        :return: Formula in CNF
        """
        # Distribute ∨ over ∧
        while True:
            old_formula = formula
            formula = re.sub(r'\(([^()]+)\s*∨\s*\((.*?)\s*∧\s*(.*?)\)\)',
                             r'((\1 ∨ \2) ∧ (\1 ∨ \3))', formula)
            if formula == old_formula:
                break
        return formula

    @staticmethod
    def convert_to_cnf(formula: str) -> str:
        """
        Convert a first-order logic formula to Conjunctive Normal Form
        :param formula: Input logical formula
        :return: Formula in CNF
        """
        # Remove universal quantifiers for simplicity
        formula = re.sub(r'∀x\s*', '', formula)

        # Eliminate implications
        formula = CNFConverter.eliminate_implications(formula)

        # Move negations inward
        formula = CNFConverter.move_negations_inward(formula)

        # Distribute OR over AND
        formula = CNFConverter.distribute_or(formula)

        return formula

class ResolutionProver:
    def __init__(self):
        """Initialize the resolution prover"""
        self.knowledge_base = set()

    def add_clause(self, clause):
        """
        Add a clause to the knowledge base
        :param clause: Clause object to add
        """
        self.knowledge_base.add(clause)

    def resolve(self, clause1, clause2):
        """
        Perform resolution between two clauses
        :param clause1: First clause
        :param clause2: Second clause
        :return: Set of resolved clauses
        """
        resolvents = set()
        for lit1 in clause1.literals:
            for lit2 in clause2.literals:
                # Check if literals can be resolved (complementary)
                if (lit1.predicate == lit2.predicate and
                    lit1.terms == lit2.terms and
                    lit1.negated != lit2.negated):
                    # Create a new clause by removing resolving literals
                    new_literals = (clause1.literals - {lit1}) | (clause2.literals - {lit2})
                    if new_literals:
                        resolvents.add(Clause(new_literals))
                    else:
                        # Empty clause means contradiction (proof found)
                        return {Clause(set())}
        return resolvents

    def prove(self, query):
        """
        Prove a query using resolution
        :param query: Clause to prove
        :return: Boolean indicating if query is provable
        """
        # Add negated query to knowledge base
        working_set = self.knowledge_base.copy()
        working_set.add(query)

        # Track generated clauses to avoid infinite loop
        generated_clauses = set()

        while True:
            new_clauses = set()
            for c1 in working_set:
                for c2 in working_set:
                    if c1 != c2:
                        resolvents = self.resolve(c1, c2)
                        new_clauses.update(resolvents)

                        # Check if empty clause (contradiction) is found
                        if Clause(set()) in new_clauses:
                            return True

            # If no new clauses can be generated, proof fails
            if new_clauses.issubset(working_set | generated_clauses):
                return False

            generated_clauses.update(new_clauses)
            working_set.update(new_clauses)

def main():
    # Create a resolution prover
    prover = ResolutionProver()

    # Original FOL statements
    fol_statements = [
        "∀x(Food(x) → LikesFood(John, x))",  # John likes all kind of food
        "Food(Apple)",  # Apple is food
        "Food(Vegetables)",  # Vegetables are food
        "∀x∀y((Eats(y,x) ∧ ¬Killed(y)) → Food(x))",  # Eating and not killed implies food
        "Eats(Anil, Peanuts)",  # Anil eats peanuts
        "∀x(Eats(Anil, x) → Eats(Harry, x))"  # Harry eats everything Anil eats
    ]

    # Convert each statement to CNF
    cnf_statements = []
    for statement in fol_statements:
        cnf = CNFConverter.convert_to_cnf(statement)
        print(f"Original: {statement}\nCNF: {cnf}\n")
        cnf_statements.append(cnf)

    # TODO: Add parsing of CNF statements to Clauses and add to prover

    # Query: John likes peanuts
    query = "¬LikesFood(John, Peanuts)"
    cnf_query = CNFConverter.convert_to_cnf(query)
    print(f"Query: {query}\nCNF Query: {cnf_query}")

    # Note: Full implementation would require parsing CNF to Clauses
    print("\nNote: Full clause parsing and resolution not implemented in this example.")

if __name__ == "__main__":
    main()

Original: ∀x(Food(x) → LikesFood(John, x))
CNF: (Food(x) → LikesFood(John, x))

Original: Food(Apple)
CNF: Food(Apple)

Original: Food(Vegetables)
CNF: Food(Vegetables)

Original: ∀x∀y((Eats(y,x) ∧ ¬Killed(y)) → Food(x))
CNF: ∀y((Eats(y,x) ∧ ¬Killed(y)) → Food(x))

Original: Eats(Anil, Peanuts)
CNF: Eats(Anil, Peanuts)

Original: ∀x(Eats(Anil, x) → Eats(Harry, x))
CNF: (Eats(Anil, x) → Eats(Harry, x))

Query: ¬LikesFood(John, Peanuts)
CNF Query: ¬LikesFood(John, Peanuts)

Note: Full clause parsing and resolution not implemented in this example.
